<a href="https://colab.research.google.com/github/obedjunias19/Applied-Natural-Language-Processing/blob/master/Chatbot_Intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [3]:
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_json('/content/drive/MyDrive/NLP Datasets/chatbot-intents.json')

In [30]:
df.head()

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'morning', 'patterns': ['Good morning'..."
2,"{'tag': 'afternoon', 'patterns': ['Good aftern..."
3,"{'tag': 'evening', 'patterns': ['Good evening'..."
4,"{'tag': 'night', 'patterns': ['Good night'], '..."


No charts were generated by quickchart


In [8]:
intents = df.intents.values

In [10]:
# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [14]:
len(tags)

232

In [18]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [26]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            print(tag)
            response = random.choice(intent['responses'])
            return response

In [29]:
chatbot('How are you')

about


"I'm Pandora, your Personal Therapeutic AI Assistant. How are you feeling today"

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [35]:
RF = RandomForestClassifier()
params = {
    'n_estimators':[10,50,100,150],
    'max_depth':[60,90,None]
}
gs = GridSearchCV(RF,params,cv=5,n_jobs=-1)

In [37]:
gs_fit = gs.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [38]:
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.934876,0.148747,0.034675,0.010622,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.382979,0.361702,0.413043,0.500000,0.413043,0.414154,0.047106,1
2,0.631271,0.093056,0.027441,0.009023,60,100,"{'max_depth': 60, 'n_estimators': 100}",0.361702,0.361702,0.434783,0.434783,0.391304,0.396855,0.032800,2
1,0.482144,0.080760,0.018147,0.003885,60,50,"{'max_depth': 60, 'n_estimators': 50}",0.340426,0.382979,0.369565,0.391304,0.347826,0.366420,0.019620,3
6,0.589802,0.061335,0.019890,0.007580,90,100,"{'max_depth': 90, 'n_estimators': 100}",0.340426,0.382979,0.304348,0.413043,0.347826,0.357724,0.037267,4
7,0.639362,0.122587,0.020792,0.000620,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.297872,0.361702,0.326087,0.413043,0.369565,0.353654,0.039285,5


In [39]:
def chatbot(model,input_text):
    input_text = vectorizer.transform([input_text])
    tag = model.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            print(tag)
            response = random.choice(intent['responses'])
            return response

In [45]:
chatbot(gs_fit,'Hi')

greeting


'Hi there. What brings you here today?'

In [43]:
chatbot(clf,'Hi')

greeting


'Great to see you. How do you feel currently?'

In [44]:
chatbot(clf,'How are you?')

about


"I'm Pandora, a Therapeutic AI Assitant designed to assist you. Tell me about yourself."

In [47]:
chatbot(gs_fit,'I am sad')

sad


"I'm here for you. Could you tell me why you're feeling this way?"

In [52]:
%%writefile app.py
counter = 0


def main():
    global counter
    st.title("Chatbot")
    st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

    counter += 1
    user_input = st.text_input("You:", key=f"user_input_{counter}")

    if user_input:
        response = chatbot(gs_fit, user_input)
        st.text_area("Chatbot:", value=response, height=100, max_chars=None, key=f"chatbot_response_{counter}")

        if response.lower() in ['goodbye', 'bye']:
            st.write("Thank you for chatting with me. Have a great day!")
            st.stop()

if __name__ == '__main__':
    main()

Writing app.py


In [51]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.199.208.139:8501

  Stopping...
  Stopping...
Exception ignored in atexit callback: <function shutdown at 0x7fec8cecf640>
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 2181, in shutdown
    h.acquire()
  File "/usr/lib/python3.10/logging/__init__.py", line 917, in acquire
    self.lock.acquire()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/bootstrap.py", line 69, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/server.py", line 397, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 308, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 798, in call_soon_threadsafe
    self._check_closed()
  File "/usr/l

In [53]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.357s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [55]:
!streamlit run /content/app.py &>/content/logs.txt &

In [56]:
!npx localtunnel --port 8501

npx: installed 22 in 3.586s
your url is: https://fine-deer-warn.loca.lt
^C
